In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install vit_keras

In [3]:
pip install tensorflow_addons

     |████████████████████████████████| 1.1 MB 5.1 MB/s 


In [5]:
!unzip /content/drive/MyDrive/Graduation_Project/CheXpert-v1.0-small.zip > /dev/null

In [1]:
import cv2
import numpy as np 
import pandas as pd

import matplotlib.pyplot as plt

import tensorflow as tf
from keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

from myowngen_v4 import DataGenerator
from vit_keras import vit, utils

In [2]:
sample_path  = 'sample_4.csv'
valid_path   = 'val_sample_2.csv' 
data_path    = "/content/"
weights_path = '/content/drive/MyDrive/Graduation_Project/model_weight_pure.hdf5'

In [3]:
train = pd.read_csv(sample_path)
#train.drop(columns=['Binary'], inplace=True)
train.head()

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,CheXpert-v1.0-small/train/patient00057/study2/...,Female,48,Frontal,AP,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1,CheXpert-v1.0-small/train/patient00060/study1/...,Female,44,Frontal,PA,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
2,CheXpert-v1.0-small/train/patient00060/study1/...,Female,44,Lateral,NaN,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
3,CheXpert-v1.0-small/train/patient00066/study1/...,Male,61,Frontal,PA,1.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
4,CheXpert-v1.0-small/train/patient00066/study1/...,Male,61,Lateral,NaN,1.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN


In [4]:
train.loc[:, train.columns[5:]] = train.loc[:, train.columns[5:]].fillna(0)

to_take = list(set(train.columns[5:])-set(['Edema', 'Atelectasis']))
train.loc[:, to_take] = train.loc[:, to_take].replace({-1:0})

train.loc[:, ['Edema', 'Atelectasis']] = train.loc[:, ['Edema', 'Atelectasis']].replace({-1:1})

In [5]:
valid = pd.read_csv(valid_path)
valid.drop('Sex_y', axis=1, inplace=True)
valid.head()

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,CheXpert-v1.0-small/train/patient04947/study3/...,Male,56,Frontal,PA,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,CheXpert-v1.0-small/train/patient38193/study3/...,Male,67,Frontal,AP,0,0,0,1,0,1,0,0,0,0,0,0,0,1
2,CheXpert-v1.0-small/train/patient47458/study7/...,Male,57,Frontal,AP,0,0,0,1,0,0,0,0,0,0,0,0,1,1
3,CheXpert-v1.0-small/train/patient38830/study3/...,Male,56,Frontal,AP,0,0,0,0,0,0,0,0,1,0,0,0,0,1
4,CheXpert-v1.0-small/train/patient26417/study1/...,Female,50,Lateral,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [6]:
# valid.loc[:, valid.columns[5:]] = valid.loc[:, valid.columns[5:]].fillna(0)

# to_take = list(set(valid.columns[5:])-set(['Edema', 'Atelectasis']))
# valid.loc[:, to_take] = valid.loc[:, to_take].replace({-1:0})

# valid.loc[:, ['Edema', 'Atelectasis']] = valid.loc[:, ['Edema', 'Atelectasis']].replace({-1:1})

In [7]:
train_generator = DataGenerator(data_path, train, 14, batch_size=32, shape=(256,256, 3), shuffle=True)

In [8]:
valid_generator = DataGenerator(data_path, valid, 14, batch_size=32, shape=(256,256, 3), shuffle=True)

In [9]:
base_model = vit.vit_b16(image_size=256,
                        pretrained=True,
                        pretrained_top = False,
                        include_top=False)

/usr/local/lib/python3.7/dist-packages/vit_keras/utils.py:83: UserWarning: Resizing position embeddings from 24, 24 to 16, 16
  UserWarning,


In [10]:
for layer in base_model.layers[:-9]:
     layer.trainable = False

for layer in base_model.layers[-9:]:
    layer.trainable = True

#building the model
model=Sequential()
model.add(base_model)
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.1))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(14,activation="softmax"))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vit-b16 (Functional)        (None, 768)               85844736  
                                                                 
 dense (Dense)               (None, 512)               393728    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               65664     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 14)                1806      
                                                                 
Total params: 86,305,934
Trainable params: 50,077,838
No

In [11]:
model.load_weights(weights_path)

In [12]:
model.layers

In [13]:
model.pop()

In [13]:
from keras.layers import Input

for layer in model.layers[:-4]:
     layer.trainable = False

for layer in model.layers[-4:]:
    layer.trainable = True

final_model = Sequential()
final_model.add(Input((256, 256, 3)))
final_model.add(model)
final_model.add(Dense(128, activation='relu'))
final_model.add(Dropout(0.1))
final_model.add(Dense(64, activation='relu'))
final_model.add(Dropout(0.1))
final_model.add(Dense(14, activation='sigmoid'))

final_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 14)                86305934  
                                                                 
 dense_3 (Dense)             (None, 128)               1920      
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 14)                910       
                                                                 
Total params: 86,317,020
Trainable params: 78,556
Non-

In [14]:
INIT_LR, EPOCHS, BS = 0.001, 5, 32
final_model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=INIT_LR),
              metrics=['binary_accuracy', tf.keras.metrics.AUC(multi_label = True)])

In [15]:
checkpoint = ModelCheckpoint(weights_path, monitor='val_auc', verbose=1, save_best_only=False, mode='auto', save_freq='epoch')
early = EarlyStopping(monitor="val_auc", mode='auto', patience=5, restore_best_weights=False)
redlr = ReduceLROnPlateau(monitor='val_auc', factor=0.1, patience=2)
callbacks_list = [checkpoint, early, redlr]

In [ ]:
history = final_model.fit(train_generator, validation_data=valid_generator, epochs=EPOCHS, batch_size=BS, verbose=1, callbacks=callbacks_list)

Epoch 1/5
1131/1883 [=================>............] - ETA: 17:34 - loss: 0.4019 - binary_accuracy: 0.8312 - auc: 0.6724

# Test

In [22]:
from sklearn.metrics import roc_auc_score

def custom_auc(y_true, y_pred):
    #labels = ['No_Finding', 'Cardiomegaly', 'Edema', 'Consolidation', 'Atelectasis', 'Pleural Effusion']
    labels = ['No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly',
       'Lung Opacity', 'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia',
       'Atelectasis', 'Pneumothorax', 'Pleural Effusion', 'Pleural Other',
       'Fracture', 'Support Devices']
       
    results = pd.DataFrame(index=labels)


    scores = []
    for i in range(14):
        score = roc_auc_score(y_true[:, i], y_pred[:, i])
        scores.append(score)
        
    results['AUC'] = scores

    return results

In [17]:
val_path   = '/content/CheXpert-v1.0-small/valid.csv'
val = pd.read_csv(val_path)

In [21]:
val.iloc[:, 5:].columns

Index(['No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly',
       'Lung Opacity', 'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia',
       'Atelectasis', 'Pneumothorax', 'Pleural Effusion', 'Pleural Other',
       'Fracture', 'Support Devices'],
      dtype='object')

In [18]:
#train_generator = DataGenerator(data_path, train, 14, batch_size=1, shape=(224,224, 3), shuffle=False)
val_generator = DataGenerator(data_path, val, 14, batch_size=1, shape=(256,256, 3), shuffle=False)

In [19]:
# actual
y_val_true   = val.iloc[:, 5:].values

# predicted
y_val_pred   = model.predict(val_generator)

results = custom_auc(y_val_true, y_val_pred)
results.AUC.mean()

0.8761556551067785

In [20]:
results

,AUC
No_Finding,0.913534
Cardiomegaly,0.787828
Edema,0.888183
Consolidation,0.920247
Atelectasis,0.839286
Pleural Effusion,0.907856


In [ ]:
# actual
y_train_true   = train.iloc[:, 5:].values

# predicted
y_train_pred   = model.predict(train_generator)

results = custom_auc(y_train_true, y_train_pred)
results.AUC.mean()

0.7247363198395431